<a href="https://colab.research.google.com/github/numberjuani/crypto_cash_carry/blob/main/Cash_and_Carry_Trade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install binance-connector
!pip3 install binance-futures-connector

In [ ]:
from binance.spot import Spot as SpotClient
from binance.delivery import Delivery as FuturesClient
import pandas as pd
%load_ext google.colab.data_table

First we establish clients to the spot, and COIN-M margined futures.

In [ ]:
spot_client = SpotClient()
futures_client = FuturesClient()

Now we obtain information for all listed contracts under the COIN-M futures, which will allows to filter by those with a valid expiration date, ignoring perpetuals.

In [ ]:
coin_m_futures_info = pd.DataFrame(futures_client.exchange_info()['symbols'])
coin_m_futures_info['deliveryDate'] = pd.to_datetime(coin_m_futures_info['deliveryDate'],unit='ms')
#remove all perpetual contracts from the list
coin_m_futures_info = coin_m_futures_info[coin_m_futures_info.contractType != 'PERPETUAL']
coin_m_futures_info = coin_m_futures_info[['symbol','contractType','marginAsset','deliveryDate']]
coin_m_futures_info

Now we obtain prices for all the futures, and filter by the contracts that do expire, which we obtained in the previous step.

In [ ]:
coin_m_futures_pricing = pd.DataFrame(futures_client.book_ticker())
non_perpetual = coin_m_futures_pricing[coin_m_futures_pricing.symbol.isin(non_perpetual_symbols)]
#since we'll be selling the futures we'll keep the bid price.
non_perpetual = non_perpetual[['symbol','bidPrice']]
non_perpetual

Now we consolidate all the futures data into one dataframe.

In [ ]:
futures = pd.merge(coin_m_futures_info,non_perpetual,on='symbol')
futures

The spot endpoint has an incredibly large number of coins, so we have to narrow it down by those that have a coin-m futures that expire and those that trade with a dollar quote.

In [ ]:
coins_to_consider = futures.marginAsset.values.tolist()
quotes_to_consider = ['USD','USDT','BUSD']

In [ ]:
spot_info = pd.DataFrame(spot_client.exchange_info()['symbols'])
spot_info = spot_info[spot_info.baseAsset.isin(coins_to_consider) & spot_info.quoteAsset.isin(quotes_to_consider)]
spot_info = spot_info[['symbol','baseAsset','quoteAsset']]
spot_info

In [ ]:
spot_prices = pd.DataFrame(spot_client.ticker_24hr())
spot_prices = spot_prices[spot_prices.symbol.isin(spot_info.symbol)]
#since the spot needs to be bought, we will save the ask price.
spot_prices = spot_prices[['symbol','askPrice']]
spot_prices

Now we merge spot info with its pricing data, like we did before.

In [ ]:
spot_info = pd.merge(spot_prices,spot_info,on='symbol')
spot_info

Now we finally are ready to merge spot and futures pairs.

In [ ]:
spot_and_futures_combo = pd.merge(spot_info,futures,left_on='baseAsset',right_on='marginAsset',suffixes=('_spot', '_future'))
spot_and_futures_combo

In [ ]:
spot_and_futures_combo['bidPrice'] = spot_and_futures_combo['bidPrice'].astype(float)
spot_and_futures_combo['askPrice'] = spot_and_futures_combo['askPrice'].astype(float)
spot_and_futures_combo['price_difference'] = spot_and_futures_combo['bidPrice'] - spot_and_futures_combo['askPrice']
spot_and_futures_combo['price_difference_pct'] = 100*(spot_and_futures_combo['price_difference']/spot_and_futures_combo['askPrice'])
spot_and_futures_combo['days_to_expiration'] = (spot_and_futures_combo.deliveryDate - pd.Timestamp.today()).dt.days
spot_and_futures_combo['return'] = spot_and_futures_combo['price_difference_pct']/spot_and_futures_combo['days_to_expiration']
spot_and_futures_combo['annual_ror'] =  365*(spot_and_futures_combo['return']/spot_and_futures_combo['days_to_expiration'])
spot_and_futures_combo.sort_values(by='annual_ror',inplace=True,ascending=False)
spot_and_futures_combo = spot_and_futures_combo[['symbol_spot','symbol_future','days_to_expiration','baseAsset','quoteAsset','bidPrice','askPrice','price_difference','price_difference_pct','return','annual_ror']]
spot_and_futures_combo